In [52]:
import json
import os
from collections import defaultdict

import gensim
from gensim.models.doc2vec import Doc2Vec
from gensim.summarization import summarize

import math
import operator
import numpy as np
from nltk import word_tokenize,sent_tokenize,pos_tag


In [90]:
def read_data(file_name):
    with open(file_name,'r',encoding="utf8") as f:
        word_vocab = set() # not using list to avoid duplicate entry
        word2vector = {}
        for line in f:
            line_ = line.strip() #Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

In [91]:
totalvocab, w2v = read_data("D:/Deep learning/pretrainedmodels/Glove vectors/glove.6B.50d.txt")

Total Words in DataSet: 400000


In [54]:
json_data=open("D:/Hacks/Wehack3/compresseddict.json").read()

compresseddict = json.loads(json_data)

In [55]:
with open("D:/Hacks/Wehack3/vocab.txt", 'r',encoding='UTF-8') as f:
    vocab = [line.rstrip('\n') for line in f]

In [56]:
len(vocab),len(compresseddict.keys())

(1216, 1216)

In [113]:
q="help mode stockpile"#stockpile->supplies
q=word_tokenize(q.lower())

for i in range(0,len(q)):
    replace=""
    mindist=1000
    if q[i].lower() not in vocab:
        vec1=w2v[q[i]]
        for j in vocab:
            if j in totalvocab:
                vec2=w2v[j]
                dist = np.linalg.norm(vec1-vec2)
                if dist<mindist:
                    mindist=dist
                    replace=j
        q[i]=replace


x=pos_tag(q)
x

[('help', 'NN'), ('mode', 'NN'), ('supplies', 'NNS')]

In [116]:
directory="D:/Hacks/Wehack3/for short testing/"
namesofdoc=os.listdir(directory)

alpha=0.50
totaldocs=len(namesofdoc)

inv_indx = defaultdict(list)

In [118]:
######## SEARCH #######
dictofdoc={}
listofdoc=[]

for i in namesofdoc:
    dictofdoc[i]=0


for i in namesofdoc:
    dictofdoc[i]=0
    listofdoc.append(i)

for i in x: 
    info=compresseddict[i[0].lower()]
    #finding number of documents query word is in
    numberofdocs=0
    numofdocsnocontext=0
    docs_counted=[]
    docs_nocontext_counted=[]
    listofdoc2=[]
    for j in info:     #j has POStag,docn,number
        listofdoc2.append(j[1])
        if j[0]==i[1]: #matches context
            if j[1] not in docs_counted: #to see whether or not the doc is already counted
                docs_counted.append(j[1])
                numberofdocs=numberofdocs+1
        else:
            if j[1] not in docs_counted: #to see whether or not the doc is already counted
                docs_nocontext_counted.append(j[1])
                numofdocsnocontext=numofdocsnocontext+1
    listofdoc=list(set(listofdoc).intersection(set(listofdoc2)))
            
            
    
    for j in info:     #j has POStag,docn,number
        try:
            if j[0]==i[1]: #matches context
                dictofdoc[j[1]]=dictofdoc[j[1]]+(alpha*j[2]*math.log10((totaldocs+1)/numberofdocs))

            else: #context not matched
                noncontext_match=j[2]
                dictofdoc[j[1]]=dictofdoc[j[1]]+((1-alpha)*j[2]*math.log10((totaldocs+1)/numofdocsnocontext))
        except:
            continue


for i in listofdoc:
    dictofdoc[i]=dictofdoc[i]*15

In [119]:
sortedd = sorted(dictofdoc.items(), key=operator.itemgetter(1))

In [120]:
listtop5=[]
for i in range(1,len(sortedd)):
    listtop5.append(sortedd[-i][0])
    if i==5:
        break

In [121]:
listtop5

['Falcon_7X-Air_Cond_and_Press44.txt',
 'Falcon_7X-Air_Cond_and_Press58.txt',
 'Falcon_7X-Air_Cond_and_Press20.txt',
 'Falcon_7X-Air_Cond_and_Press14.txt',
 'Falcon_7X-Air_Cond_and_Press60.txt']

In [126]:
dirimg="D:/Hacks/Wehack3/Tesseract work/"
images=os.listdir(dirimg)

sendtonid={}
for i in listtop5:
    try:
        with open(directory+i, 'r',encoding='UTF-8') as f:
            s=f.read()
    except:
        with open(directory+i, 'r') as f:
            s=f.read()
    listofimg=[]
    for j in images:
        if j.startswith(i[:-4]):
            listofimg.append(j)
    if len(listofimg)==0:
        listofimg="k.jpg"
    sendtonid[i]=[summarize(s),listofimg,i]

In [127]:
i[:-4]

'Falcon_7X-Air_Cond_and_Press60'

In [128]:
sendtonid

{'Falcon_7X-Air_Cond_and_Press44.txt': ['select  one  of  the  five  following  modes:  NORMAL,  BACKUP,  CREW  OFF,  PAX  OFF  and \n-  In MAN mode a desired cabin hot air valve position.\nIn  MAN  mode,  PAX  AFT  TEMP  rotary  knob  is  disabled  (INOP  lighted)  and  PAX  valve \nposition setting is made through the PAX FWD TEMP rotary pushbutton.\nIn ECS EMERG mode, the AMSAC modulates the cabin hot air valve to the desired position \nMAN mode (the PAX AFT TEMP is not used by the AMSEC).\nThe XBLEED ECS pushbutton allows the crew members to manually open or close the ECS \nThe  AMSAC  drives  the  valve  when  the  ECS  Mode  Selector  is  in  the \nNORMAL, PAX OFF, CREW OFF or BACKUP position.\nThe AMSAC automatically drives the XBLEED ECS valve closed if \nWhen the PACK rotary switch is in the EMERG position, the AMSEC drives the XBLEED \nopen position, except in EMERG mode.\nThe  GND VENT  pushbutton  allows  the  crew  members  to  manually  close  the  recirculation \n-  The 